# Analysis of outputs

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
import math
import numpy as np
from textwrap import wrap
import sys
# if ".." not in sys.path:
#     sys.path.insert(0, "..")
# from analysis.analysis_data_processing.analysis_population_df_processing import (
#     create_population_df,
# )
# from analysis.analysis_population_df_processing.redaction import(
#     redact_and_round_df,
#     redact_and_round_column,
#     further_redaction,
#     further_redaction_all,
#     redact_to_five_and_round
# )
# from analysis.analysis_population_df_processing.analysis_breakdowns import analysis_breakdowns

In [ ]:
# Create dictionaries of oximetry, blood pressure and proactive care codes:
# Keys are SNOMED codes, values are the terms they refer to
oximetry_codes_df = pd.read_csv("../../codelists/opensafely-pulse-oximetry.csv")
oximetry_codes_dict = oximetry_codes_df.set_index("code")["term"].to_dict()
bp_codes_dict = {
    413606001: "Average home systolic blood pressure",
    314446007: "Average day interval systolic blood pressure",
    413605002: "Average home diastolic blood pressure",
    314461008: "Average day interval diastolic blood pressure",
}
proactive_codes_dict = {934231000000106: "Provision of proactive care"}


# Create dictionary of oximetry, blood pressure and proactive care headers:
# Keys are oximetry headers in input csv files (i.e. pulse_oximetry_code),
# values are the terms they refer to
oximetry_headers_dict = {
    f"healthcare_at_home_{k}": v for k, v in oximetry_codes_dict.items()
}
bp_headers_dict = {f"healthcare_at_home_{k}": v for k, v in bp_codes_dict.items()}
proactive_headers_dict = {
    f"healthcare_at_home_{k}": v for k, v in proactive_codes_dict.items()
}

oximetry_codes = pd.read_csv("../../codelists/opensafely-pulse-oximetry.csv",usecols=["code"])


In [ ]:
def create_population_df(homecare_type: str, dir: str) -> pd.DataFrame:
    """Function to create population data frame for a particular homecare type
    which includes all weeks and create a dictionary of cohort size for each
    individual week"""
    # find the input csv files
    filepaths = [
        f
        for f in os.listdir(dir)
        if (f.startswith(f"input_{homecare_type}") and f.endswith(".csv"))
    ]
    # append the directory path to filename
    filepaths_dir = [dir + filepath for filepath in filepaths]

    filepaths_dir

    # create empty list to append dataframes
    dfs = []

    for file in filepaths_dir:
        # read in files
        output = pd.read_csv(file)
        # Add the index date to the file by extracting index from filename
        output["index_date"] = pd.to_datetime(
            file.split("_",)[4].split(
                ".csv"
            )[0],
            dayfirst=True,
        )
        # Append the dataframes to the list
        dfs.append(output)

    # Combine all the dataframes together
    population_df = pd.concat(dfs)

    return population_df

def homecare_type_dir(homecare_type: str) -> Dict[str, str]:
    """Function to return a dictionary containing the input directory
    (location of the relevant input csv files) and output directory (where to
    store the analysis outputs) for a specific homecare type"""
    return dict(
        input_dir=f"../../output/{homecare_type}/0.2_join_cohorts/",
        output_dir=f"../../output/{homecare_type}/0.3_analysis_outputs/",
    )

In [ ]:
def redact_and_round_column(column: pd.Series) -> pd.Series:
    """Function which takes a column of data, redacts any values less than or
    equal to 5 and rounds all other values up to nearest 5"""
    # New column variable will contain the new values with any necessary
    # redacting and rounding applied
    new_column = []
    # For loop to apply redacting and rounding to all integer or float values
    # in the column
    for value in column:
        if type(value) == int or type(value) == float:
            # Redact values less than or equal to 5
            if value <= 5:
                value = "[REDACTED]"
            # Round all values greater than 5 up to nearest 5
            else:
                value = int(5 * math.ceil(float(value) / 5))
        # Resulting value is added to the new column
        new_column.append(value)
    return new_column
    

def redact_and_round_df(df: pd.DataFrame) -> pd.DataFrame:
    """Function to take a dataframe, redact any values less than or equal to 5 and
    round all other values up to nearest 5"""
    # Apply redacting and rounding to each column of the dataframe
    for column in df.columns.values:
        df[column] = redact_and_round_column(df[column])
    return df

In [ ]:
homecare_type = "oximetry"
dirs = homecare_type_dir(homecare_type)
population_df = create_population_df(homecare_type, dirs["input_dir"])

# Test cases
population_df.loc[0,"age"] = 0
population_df.loc[1,"age"] = 39
population_df.loc[2,"age"] = 40
population_df.loc[3,"age"] = 41
population_df.loc[4,"age"] = 49
population_df.loc[5,"age"] = 50
population_df.loc[6,"age"] = 51
population_df.loc[7,"age"] = 64
population_df.loc[8,"age"] = 65
population_df.loc[9,"age"] = 66

# Add age_group column
population_df.loc[population_df['age']<=39, 'age_group'] = 'Age 0-39'
population_df.loc[population_df['age'].between(40,49), 'age_group'] = 'Age 40-49'
population_df.loc[population_df['age'].between(50,64), 'age_group'] = 'Age 50-64'
population_df.loc[population_df['age']>=65, 'age_group'] = 'Age 65 or over'

test = population_df.loc[:,["age", "age_group"]]

# sum_df = population_df.groupby("index_date")["patient_id"].nunique().to_frame()
# sum_df = redact_and_round_df(sum_df)



## Data Processing

In [ ]:
# Create dictionary of oximetry codes: keys are SNOMED codes, values are the terms they refer to
oximetry_codes_df = pd.read_csv('../../codelists/opensafely-pulse-oximetry.csv')
oximetry_codes_dict = oximetry_codes_df.set_index("code")["term"].to_dict()
# Create dictionary of oximetry headers:
# Keys are oximetry headers in input csv files (i.e. pulse_oximetry_code), values are the terms they refer to
oximetry_headers_dict = {f"pulse_oximetry_{k}":v for k,v in oximetry_codes_dict.items()}



## Create dataframe of sums of pulse oximetry codes for each index week

In [ ]:
# # Create population data frame which includes all weeks and dictionary of cohort size for each individual week
# population_df, cohort_size = create_population_df("../../output/")

# # Create lists of current and required headers
# # Convert pulse oximetry codelist csv into data frame
# oximetry_codes_df = pd.read_csv('../../codelists/opensafely-pulse-oximetry.csv')
# # Extract list of SNOMED codes
# oximetry_codes_list = oximetry_codes_df['code'].tolist()
# # List of pulse oximetry headers in population dataframe
# oximetry_codes_headers = [f'pulse_oximetry_{x}' for x in oximetry_codes_list]
# # List of headers using descriptions as required
# oximetry_headers = oximetry_codes_df['term'].tolist()

# # Create dictionary for renaming oximetry headers
# oximetry_dictionary = {}
# for n in range(0,len(oximetry_codes_df)):
#     oximetry_dictionary[oximetry_codes_headers[n]] = oximetry_headers[n]

# #Create data frame of sum totals for each index date for each oximetry code
# oximetry_sum = population_df.groupby(['index_date'], as_index=False)[oximetry_codes_headers].sum()

# # Rename oximetry headers in oximetry sums data frame
# oximetry_sum.rename(columns=oximetry_dictionary,inplace=True)

# # Save the dataframe in outputs folder
# #oximetry_sum.to_csv('../../output/oximetry_sums.csv') 


## Test different possibilities for redacting and rounding functions

In [ ]:
def redact_to_five_and_round(counts_df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """Function which determines for each index date if any value in a dataframe column
    is <= 5 and if so redacts all values <=5 then continues redacting the next lowest
    value until the redacted values add up to >= 5.
    All remaining values are then rounded up to nearest 5"""
    # For each index date
    for index_date in counts_df.index_date.unique():
        # Create temporary dataframe of all the rows with that index date
        temp_df = counts_df[counts_df["index_date"] == index_date]
        # If sum of values in the column <= 5
        if pd.to_numeric(temp_df[column_name], errors="coerce").sum() <= 5:
            # Redact all values in the column
            temp_df[column_name][temp_df[column_name] != 0] = "[REDACTED]"
        # Else if there are any non-zero numbers <= 5 in the column of interest
        elif (
            pd.to_numeric(
                temp_df[column_name][
                    (pd.to_numeric(counts_df["counts"], errors='coerce') <= 5) & (pd.to_numeric(counts_df["counts"], errors='coerce') != 0)
                ],
                errors="coerce",
            ).count()
            > 0
        ):
            # Store total quantity redacted
            total_redacted = 0
            # For each row
            for index in temp_df.index.values:
                # If column value is non-zero and less than 5
                if ((pd.to_numeric(temp_df.loc[index, column_name], errors="coerce") <= 5) 
                    & ((pd.to_numeric(temp_df.loc[index, column_name], errors="coerce") <= 5) != 0)):
                    # Add to the total_redacted variable
                    total_redacted += temp_df.loc[index, column_name]
                    # Redact the value
                    temp_df.loc[index, column_name] = "[REDACTED]"
                    # While total_redacted <= 5
                    while total_redacted <= 5:
                        # Find index of the lowest non-zero non-redacted count for that index date
                        min_index = pd.to_numeric(
                            temp_df[(temp_df[column_name] != "[REDACTED]") & (temp_df[column_name] != "[REDACTED]")][column_name]
                        ).idxmin()
                        # Add to the total_redacted variable
                        total_redacted += temp_df.loc[min_index, column_name]
                        # Redact the value
                        temp_df.at[min_index, column_name] = "[REDACTED]"
        # Update counts dataframe with the redactions
        counts_df.update(temp_df)
        # Round all numeric values in column up to nearest 5
        for index in counts_df.index.values:
            value = counts_df.loc[index, column_name]
            if type(value) == int or type(value) == float:
                counts_df.loc[index, column_name] = int(5 * math.ceil(float(value) / 5))
    return counts_df

In [ ]:
code = "1325191000000108"
column_name = "region"
codes_dict = oximetry_codes_dict
variable_title = "Test Title"
term = codes_dict[int(code)]

# Population of interest is all patients with the code
codes_df = population_df.loc[population_df[term] == 1]
# Count the number of patients in each age group for each index date
counts_df = codes_df.groupby(["index_date", column_name]).size().reset_index()
counts_df.rename(columns={0: "counts"}, inplace=True)

# Count the denominator (the total size of the cohort for each week)
counts_df["denominators"] = counts_df["index_date"].map(
    (codes_df.groupby("index_date").size()).to_dict()
)

# Test cases
# One zero value <= 5 0-8
counts_df.iloc[[6,7], 2] = [10, 0]
# Nothing <= 5 9-17
# One non-zero value <=5 18 - 26
counts_df.iloc[21, 2] = 5
# Multiple values <=5 totalling <5 (no zeroes) 27 - 35
counts_df.iloc[[31,32], 2] = [5,10]
# Multiple values <=5 totalling <=5 (some zeroes) 36 - 44
counts_df.iloc[[36, 39, 41, 43], 2] = [1,0,4,0]
# Multiple zero values, all others >5 45 - 53
counts_df.iloc[[47, 49, 50], 2] = 0
# Sum of values <=5 (no zeroes) 54 - 62
counts_df.iloc[[58, 54,62, 60], 2] = [1,2,1,1]
# Sum of values <=5 (all zeroes) 63 - 71
counts_df.iloc[[64, 67, 68, 71], 2] = 0
# Sum of values <=5 (some zeroes) 72 - 80
counts_df.iloc[[72, 74, 75, 76, 77], 2] = [1,1,0,1,0]

#32,58, 78 (or 80)
# 21,25,31,35,36,41,42,54,58,59,60,62,72,74,76,78

# Apply redacting and rounding to the counts
counts_df = redact_to_five_and_round(counts_df, "counts")
#counts_df.iloc[36:45,]
